## Working with LINQ

### Item 29: Prefer Iterator Methods to Returning Collections


In [1]:
public static IEnumerable<char> GenerateAlphabet() {
    var letter = 'a';
    while (letter <= 'z') {
        yield return letter;
        letter++;
    }
}

In [3]:
using System;

foreach(char c in GenerateAlphabet()) {
    Console.Write(c + " ");
}

a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [4]:
var allNumbers = Enumerable.Range(0, int.MaxValue);

TThis generate-as-needed strategy highlights one other important idiom when writing iterator methods. Iterator methods create an object that knows how to generate the sequence. The code to generate the sequence executes only when a caller requests an item in the sequence. That means very little of the code executes when a generator method is called.

### Item 30: Prefer Query Syntax to Loops

In [8]:
// looping approach
var foo = new int[11];
for (var num=0; num<foo.Length; num++) {
    foo[num] = num * num;
}

foreach(var i in foo) {
    Console.Write(i + " ");
}

0 1 4 9 16 25 36 49 64 81 100 

In [18]:
// query syntax
var foo = (from n in Enumerable.Range(0, 11) select n * n).ToArray();
Array.ForEach(foo, n => Console.Write(n + " "));

0 1 4 9 16 25 36 49 64 81 100 

We can create our own ForAll implementation...

In [ ]:
public static class Extensions { // will need to do this outside of jupyter...
    public static void ForAll<T>(this IEnumerable<T> sequence, Action<T> action) {
        foreach(T item in sequence) {
            action(item);
        }
    }
}

Change the problem to generating only those pairs where the sum of X and Y is less than 5.

In [26]:
private static IEnumerable<Tuple<int, int>> ProduceIndices() {
    for (var x = 0; x < 100; x++)
        for (var y = 0; y < 100; y++)
            if (x + y < 5)
                yield return Tuple.Create(x, y);
}

var indices = ProduceIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


In [27]:
private static IEnumerable<Tuple<int, int>> QueryIndices() {
    return from x in Enumerable.Range(0, 100)
            from y in Enumerable.Range(0, 100)
            where x + y < 5
            select Tuple.Create(x, y);
}

var indices = QueryIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


### Item 31: Create Composable APIs for Sequences
To see the benefits of writing iterator methods, let’s take a simple example and examine the translation.

In [31]:
using System;
using System.Collections.Generic;

public static void Unique(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            Console.WriteLine(num);
        }
    }
}

In [32]:
// suppose that you instead wrote it this way...
public static IEnumerable<int> UniqueV2(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            yield return num;
        }
    }
}

In [35]:
var nums = new int[] {1, 2, 2, 3, 4, 5, 5};
foreach(var num in UniqueV2(nums)) {
    Console.Write(num + " ");
}

1 2 3 4 5 

### Item 32: Decouple Iterations from Actions, Predicates, and Functions

In [11]:
// you might write like this

public List<int> CreateSequence(int numberOfElements, int startAt, int stepBy) {
    var collection = new List<int>(numberOfElements);
    for (int i=0; i<numberOfElements; i++) {
        collection.Add(startAt + i*stepBy);
    }
    return collection;
}   


var myList = CreateSequence(5, 2, 10);
foreach(var element in myList) {
    Console.Write(element + " ");
}

2 12 22 32 42 

In [12]:
// you can remove limitations by making the generation function an iterator method
public IEnumerable<int> CreateSequence2(int numberOfElements, int startAt, int stepBy) {
    for(var i=0; i<numberOfElements; i++) {
        yield return startAt + i * stepBy;
    }
}


var myList = CreateSequence2(5, 2, 10);
foreach(var element in myList) {
    Console.Write(element + " ");
}

2 12 22 32 42 

### Item 34: Loosen Coupling by Using Function Parameters
Creating interfaces and coding against them results in looser coupling than does relying on base classes. There are only two important differences: First, using an interface does not enforce any class hierarchy on your users. But second, you can’t easily provide a default implementation for any of the behavior necessary for client code.

Do you really need to define an interface? Or will a more loosely coupled approach, such as defining a delegate signature, be better?

In [18]:
// one approach
public static IEnumerable<string> Zip(IEnumerable<string> first, IEnumerable<string> second) {
    using (var firstSequence = first.GetEnumerator()) {
        using (var secondSequence = second.GetEnumerator()) {
            while (firstSequence.MoveNext() && secondSequence.MoveNext()) {
                yield return string.Format("{0}{1}", firstSequence.Current, secondSequence.Current);
            }
        }
    }
}

var l1 = new List<string>{"1", "2", "3"};
var l2 = new List<string>{"4", "5", "6"};

foreach(var item in Zip(l1, l2)) {
    Console.Write(item + " ");
}

14 25 36 

In [23]:
// alternatively
public static IEnumerable<string> Zip2(
    IEnumerable<string> first,
    IEnumerable<string> second,
    Func<string, string, string> zipper
) {
     using (var firstSequence = first.GetEnumerator()) {
         using (var secondSequence = second.GetEnumerator()) {
             while (firstSequence.MoveNext() && secondSequence.MoveNext()) {
                 yield return zipper(firstSequence.Current, secondSequence.Current);
             }
         }
     }
}


var result = Zip2(l1, l2, (one, two) => string.Format("{0}{1}", one, two));
foreach(var item in result) {
    Console.Write(item + " ");
}

14 25 36 

### Item 35: Never Overload Extension Methods

### Item 36: Understand How Query Expressions Map to Method Calls

LINQ is built on two concepts: a query language, and a translation from that query language into a set of methods. The C# compiler converts query expressions written in that query language into method calls.

In [21]:
var numbers = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var smallNumbers = from n in numbers where n < 5 select n;

foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 

In [23]:
smallNumbers = numbers.Where(n => n < 5);
foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 

The select clause is removed because it is redundant. That’s safe because the select operates on an immediate result from another query expression (in this example, where). When the select does not operate on the immediate result of another expression, it cannot be optimized away. Consider this query:

In [24]:
var allNumbers = from n in numbers select n;
foreach(var n in allNumbers) {
    Console.Write(n + " ");
}

0 1 2 3 4 5 6 7 8 9 

In [25]:
// it'll be translated into this method call...
allNumbers = numbers.Select(n => n);

Note that select is often used to transform or project one input element into a different element or into a different type.

In [26]:
var numbers = new int[]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var smallNumbers = from n in numbers where n < 5 select n * n;
foreach(var n in smallNumbers) {
    Console.Write(n + " ");
}

0 1 4 9 16 

In [31]:
// or you could transform it into a different type
var squares = from n in numbers where n < 5
    select new {Number=n, Square=n*n};

foreach(var item in squares) {
    Console.WriteLine(item);
    Console.WriteLine(item.Number + " " + item.Square);
}

{ Number = 0, Square = 0 }
0 0
{ Number = 1, Square = 1 }
1 1
{ Number = 2, Square = 4 }
2 4
{ Number = 3, Square = 9 }
3 9
{ Number = 4, Square = 16 }
4 16


In [56]:
class Employee {
    public readonly string FirstName;
    public readonly string LastName;
    public readonly int Age;
    
    public Employee(string FirstName, string LastName, int Age) {
        this.FirstName = FirstName;
        this.LastName = LastName;
        this.Age = Age;
    }
    
    public override string ToString() {
        return $"{this.FirstName} {this.LastName} {this.Age}";
    }
}

List<Employee> employees = new List<Employee>() {
    new Employee("Bob", "Builder", 32),
    new Employee("Jane", "Amber", 33),
    new Employee("Mad", "Max", 22),
    new Employee("Danny", "Boy", 22),
};

foreach(var employee in employees) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33
Mad Max 22
Danny Boy 22


In [57]:
var people = from e in employees
    where e.Age > 30
    orderby e.LastName, e.FirstName, e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Jane Amber 33
Bob Builder 32


In [58]:
// not correct, sorts the entire sequence three times...
var people = from e in employees
    where e.Age > 30
    orderby e.LastName
    orderby e.FirstName
    orderby e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33


In [59]:
// we also have the option of using descending...
var people = from e in employees
    where e.Age > 30
    orderby e.LastName descending, e.FirstName, e.Age
    select e;

foreach(var employee in people) {
    Console.WriteLine(employee);
}

Bob Builder 32
Jane Amber 33


In [60]:
var results = from e in employees
    group e by e.Age into d
    select new 
    {
        Department = d.Key,
        Size = d.Count()
    };

foreach(var item in results) {
    Console.WriteLine(item);
}

{ Department = 32, Size = 1 }
{ Department = 33, Size = 1 }
{ Department = 22, Size = 2 }


The final methods to understand are the SelectMany, Join and GroupJoin.

In [65]:
int[] odds = {1, 3, 5, 7};
int[] evens = {2, 4, 6, 8};

var pairs = from oddNumber in odds
    from evenNumber in evens
    select new {
        oddNumber,
        evenNumber,
        Sum = oddNumber + evenNumber
    };

foreach(var item in pairs) {
    Console.WriteLine(item);
}

{ oddNumber = 1, evenNumber = 2, Sum = 3 }
{ oddNumber = 1, evenNumber = 4, Sum = 5 }
{ oddNumber = 1, evenNumber = 6, Sum = 7 }
{ oddNumber = 1, evenNumber = 8, Sum = 9 }
{ oddNumber = 3, evenNumber = 2, Sum = 5 }
{ oddNumber = 3, evenNumber = 4, Sum = 7 }
{ oddNumber = 3, evenNumber = 6, Sum = 9 }
{ oddNumber = 3, evenNumber = 8, Sum = 11 }
{ oddNumber = 5, evenNumber = 2, Sum = 7 }
{ oddNumber = 5, evenNumber = 4, Sum = 9 }
{ oddNumber = 5, evenNumber = 6, Sum = 11 }
{ oddNumber = 5, evenNumber = 8, Sum = 13 }
{ oddNumber = 7, evenNumber = 2, Sum = 9 }
{ oddNumber = 7, evenNumber = 4, Sum = 11 }
{ oddNumber = 7, evenNumber = 6, Sum = 13 }
{ oddNumber = 7, evenNumber = 8, Sum = 15 }


In [67]:
var values = odds.SelectMany( oddNumber => evens,
    (oddNumber, evenNumber) => 
    new {
        oddNumber,
        evenNumber,
        Sum = oddNumber + evenNumber
    }
);

foreach(var item in values) {
    Console.WriteLine(item);
}

{ oddNumber = 1, evenNumber = 2, Sum = 3 }
{ oddNumber = 1, evenNumber = 4, Sum = 5 }
{ oddNumber = 1, evenNumber = 6, Sum = 7 }
{ oddNumber = 1, evenNumber = 8, Sum = 9 }
{ oddNumber = 3, evenNumber = 2, Sum = 5 }
{ oddNumber = 3, evenNumber = 4, Sum = 7 }
{ oddNumber = 3, evenNumber = 6, Sum = 9 }
{ oddNumber = 3, evenNumber = 8, Sum = 11 }
{ oddNumber = 5, evenNumber = 2, Sum = 7 }
{ oddNumber = 5, evenNumber = 4, Sum = 9 }
{ oddNumber = 5, evenNumber = 6, Sum = 11 }
{ oddNumber = 5, evenNumber = 8, Sum = 13 }
{ oddNumber = 7, evenNumber = 2, Sum = 9 }
{ oddNumber = 7, evenNumber = 4, Sum = 11 }
{ oddNumber = 7, evenNumber = 6, Sum = 13 }
{ oddNumber = 7, evenNumber = 8, Sum = 15 }


In [70]:
var numbers = new int[] {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
var labels = new string[] {"0", "1", "2", "3", "4", "5"};

var query = from num in numbers
    join label in labels on num.ToString() equals label
    select new {num, label};

foreach(var item in query) {
    Console.WriteLine(item);
}

{ num = 0, label = 0 }
{ num = 1, label = 1 }
{ num = 2, label = 2 }
{ num = 3, label = 3 }
{ num = 4, label = 4 }
{ num = 5, label = 5 }


### Item 37: Prefer Lazy Evaluation to Eager Evaluation in Queries
Let's start with an example to explain the difference between lazy and eager evaluation.

In [10]:
using System;
using System.Threading;


public class Example {
    public static IEnumerable<TResult> Generate<TResult>(int number, Func<TResult> generator) {
        for (var i=0; i<number; i++) {
            yield return generator();
        }
    }
    
    public static void LazyEvaluation() {
        Console.WriteLine($"Start time for Test One: {DateTime.Now:T}");
        var sequence = Generate(10, () => DateTime.Now);
        Console.WriteLine("Waiting....");
        Thread.Sleep(1000);
        
        Console.WriteLine("Iterating...");
        foreach (var value in sequence) {
            Console.WriteLine($"{value:T}");
        }
        
        Console.WriteLine("Waiting....\t");
        Thread.Sleep(1000);
        
        Console.WriteLine("Iterating...");
        foreach(var value in sequence) {
            Console.WriteLine($"{value:T}");
        }
    }
}

Example.LazyEvaluation();

Start time for Test One: 4:44:09 PM
Waiting....
Iterating...
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
4:44:10 PM
Waiting....	
Iterating...
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM
4:44:11 PM


In this example of lazy evaluation, notice that the sequence is generated each time it is iterated, as evidenced by the different time stamps. The sequence variable does not hold the elements created. Rather, it holds the expression tree that can create the sequence.

In [13]:
var sequence1 = Example.Generate(4, () => DateTime.Now);
var sequence2 = from value in sequence1 select value.ToUniversalTime();

foreach(var item in sequence2) {
    Console.WriteLine(item);
}

26/06/2021 6:44:47 AM
26/06/2021 6:44:47 AM
26/06/2021 6:44:47 AM
26/06/2021 6:44:47 AM


Sequence 1 and sequence 2 share functional composition, not data. Sequence 2 is not built by enumerating the values in sequence 1 and modifying each value. Rather, it is created by executing the code that produces sequence 1, followed by the code that produces sequence2.

Consider this small program...

In [2]:
static IEnumerable<int> AllNumbers() {
    var number = 0;
    while (number < int.MaxValue) {
        yield return number++;
    }
}

public static void AnotherExample() {
    var answers = from number in AllNumbers() select number;
    var smallNumbers = answers.Take(10);
    foreach(var num in smallNumbers) {
        Console.Write(num + " ");
    }
}

AnotherExample();

0 1 2 3 4 5 6 7 8 9 

This sample illustrates what I mean about a method that does not need the full sequence. The reason this works as quickly as it does is that the entire sequence is not needed. The Take() method returns the first N objects from the sequence, so nothing else matters. You can rewrite this query to run forever...

For example, the following two queries produce the same result. However, the second query may execute faster. Sophisticated providers will optimize the query, and both queries will have the same performance metrics. However, in the LINQ to Objects implementation (provided by System.Linq.Enumerable), all products are read and sorted.

In [7]:
public class Product {
    public readonly string Name;
    public int UnitsInStock {get; set;}
    
    public Product(string Name, int UnitsInStock) {
        this.Name = Name;
        this.UnitsInStock = UnitsInStock;
    }
    
    public override string ToString() {
        return this.Name + ": " + this.UnitsInStock.ToString();
    }
}

var product = new Product("Toothbrush", 11);
Console.WriteLine(product.ToString());

Toothbrush: 11


In [9]:
var products = new List<Product> {
    new Product("Toothbrush", 11),
    new Product("Floss", 7),
    new Product("Shampoo", 3),
    new Product("Soap", 17),
    new Product("Nail Clippers", 1)
};

In [10]:
var sortedProductsSlow = from p in products
    orderby p.UnitsInStock descending
    where p.UnitsInStock > 10
    select p;
    
foreach(var item in sortedProductsSlow) {
    Console.WriteLine(item);
}

Soap: 17
Toothbrush: 11


In [11]:
// Filter before order
var sortedProductsFast = from p in products
    where p.UnitsInStock > 10
    orderby p.UnitsInStock descending
    select p;

foreach(var item in sortedProductsFast) {
    Console.WriteLine(item);
}

Soap: 17
Toothbrush: 11


Notice that the first query sorts the entire series and then throws away any products whose total in stock is less than 100. The second query filters the sequence first, resulting in a sort on what may be a much smaller sequence. You need to understand which methods require the full sequence, and try to execute those last in your query expression.

At other times, though, you do want a snapshot of the values taken at a point in time. There are two methods you can use to generate the sequence immediately and store the results in a container: ToList() and ToArray().

In [17]:
public static IEnumerable<int> GetItems() {
    for (int i=1; i<=10; i++) {
        yield return i;
    }
}

In [20]:
foreach(var item in GetItems()) {
    Console.Write(item + " ");
}

1 2 3 4 5 6 7 8 9 10 

In [24]:
var items = GetItems().ToList();
foreach(var item in items) {
    Console.Write(item + " ");
}

1 2 3 4 5 6 7 8 9 10 

In [25]:
var items = GetItems().ToArray();
foreach(var item in items) {
    Console.Write(item + " ");
}

1 2 3 4 5 6 7 8 9 10 

In almost all cases, lazy evaluation saves work and is more versatile than eager evaluation. In the rare cases when you do need eager evaluation, you can force it by running the query and storing the sequence results using ToList() or ToArray().

### Item 38: Prefer Lambda Expressions to Methods
The following code snipped has the same logic repeated several times.

In [28]:
public class Employee {
    public readonly string FirstName;
    public readonly string LastName;
    public double Salary {get; set;}
    public double YearsOfService {get; set;}
    
    public Employee (string FirstName, string LastName, double Salary, double YearsOfService) {
        this.FirstName = FirstName;
        this.LastName = LastName;
        this.Salary = Salary;
        this.YearsOfService = YearsOfService;
    }
    
    public override string ToString() {
        return this.FirstName + this.LastName + " " + this.YearsOfService.ToString() + " " + this.Salary.ToString();
    }
}

var employee = new Employee("Bob", "Brozman", 60000, 10);
Console.WriteLine(employee.ToString());

BobBrozman 10 60000


In [31]:
var allEmployees = new List<Employee> {
    new Employee("Bob", "Brozman", 60000, 10),
    new Employee("Sharon", "Smith", 40000, 5),
    new Employee("Betty", "White", 120000, 25),
    new Employee("Simon", "Grimes", 75000, 8),
};

foreach(var employee in allEmployees) {
    Console.WriteLine(employee);
}

BobBrozman 10 60000
SharonSmith 5 40000
BettyWhite 25 120000
SimonGrimes 8 75000


In [35]:
var earlyFolks = from e in allEmployees
    where e.YearsOfService >= 10
    where e.Salary < 100000
    select e;

foreach(var employee in earlyFolks) {
    Console.WriteLine(employee);
}

BobBrozman 10 60000


### Item 39: Avoid Throwing Exceptions in Functions and Actions
When you create code that executes over a sequence of values and the code throws an exception somewhere in that sequence processing, you'll have problems recovering state. You don’t know how many elements were processed, if any. You don't know what needs to be rolled back. You can't restore the program state at all.

You fix this situation by guaranteeing that whenever the method does not complete, the observable program state does not change. You can implement this in various ways, each with its own benefits and risks.

The first and easiest approach is to rework the action so that you can ensure that the action method, expressed earlier in the lambda expression, never throws an exception.

Sometimes you may not be able to guarantee that those expressions never throw an exception. Now you must take more-expensive defensive measures. You need to rework the algorithm to take into account the possibility of an exception. That means doing all the work on a copy and then replacing the original sequence with the copy only if the operation completes successfully.

### Item 40: Distinguish Early from Deferred Execution


### Item 41: Avoid Capturing Expensive Resources
When you capture a variable in a closure, the object referenced by that variable can have its lifetime extended. It is not garbage until the last delegate referencing that captured variable becomes garbage. Under some circumstances it may last even longer.


### Item 42: Distinguish between IEnumerable and IQueryable Data Sources
IQueryable<T> and IEnumerable<T> have very similar API signatures. IQueryable<T> derives from IEnumerable<T>. You might think that these two interfaces are interchangeable. In many cases they are, and that’s by design. In contrast, a sequence is a sequence, but sequences are not always interchangeable. Their behaviors are different, and their performance metrics can be very, very different.

In [39]:
public class Customer {
    public readonly string Name;
    public readonly string City;
    
    public Customer(string Name, string City) {
        this.Name = Name;
        this.City = City;
    }
    
    public override string ToString() {
        return this.Name + ", " + this.City;
    }
}

var customer = new Customer("Simon", "Wellington");
Console.WriteLine(customer);

Simon, Wellington


In [42]:
var customers = new List<Customer> {
    new Customer("Simon", "Wellington"),
    new Customer("Jane", "New York"),
    new Customer("Jack", "Sydney"),
    new Customer("Jim", "Sydney"),
    new Customer("Vlad", "Moscow")
};

foreach(var customer in customers) {
    Console.WriteLine(customer);
}

Simon, Wellington
Jane, New York
Jack, Sydney
Jim, Sydney
Vlad, Moscow


In [44]:
var q = from c in customers
    where c.City == "Sydney"
    select c;

var finalAnswer = from c in q
    orderby c.Name
    select c;

foreach(var customer in finalAnswer) {
    Console.WriteLine(customer);
}

Jack, Sydney
Jim, Sydney


In [45]:
var q = (from c in customers
    where c.City == "Sydney"
    select c).AsEnumerable();

var finalAnswer = from c in q
    orderby c.Name
    select c;

foreach(var customer in finalAnswer) {
    Console.WriteLine(customer);
}

Jack, Sydney
Jim, Sydney


In the second case, returning the first query as an IEnumerable<T> sequence means that subsequent operations use the LINQ to Objects implementation and are executed using delegates. You should care about the differences because many queries work quite a bit more efficiently if you use IQueryable functionality than if you use IEnumerable functionality.

### Item 43: Use Single() and First() to Enforce Semantic Expectations on Queries
Single() returns exactly one element. If no elements exist, or if multiple elements exist, then Single() throws an exception. That’s a rather strong statement about your expectations. Yes, it fails if your assumptions are wrong, but it fails quickly and in a way that doesn’t cause any data corruption.

In [51]:
public class Person {
    public string FirstName {get; set;}
    public string LastName {get; set;}
    
    public override string ToString() {
        return this.FirstName + " " + this.LastName;
    }
}

var somePeople = new List<Person> {
    new Person { FirstName = "Bill", LastName = "Gates"},
    new Person { FirstName = "Simon", LastName = "Smith"},
    new Person { FirstName = "Bill", LastName = "Johnson"}
};

foreach(var person in somePeople) {
    Console.WriteLine(person);
}

Bill Gates
Simon Smith
Bill Johnson


In [53]:
var answer = (from p in somePeople
    where p.FirstName == "Simon"
    select p).Single();

Console.WriteLine(answer);

Simon Smith


When there is more than one element in the sequence an exception will be thrown.

In [55]:
var answer = (from p in somePeople
    where p.FirstName == "Bill"
    select p).Single();

Unhandled exception: System.InvalidOperationException: Sequence contains more than one element
   at System.Linq.ThrowHelper.ThrowMoreThanOneElementException()
   at System.Linq.Enumerable.Single[TSource](IEnumerable`1 source)
   at Submission#58.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

It'll also throw an exception if no elements are found...

In [57]:
var answer = (from p in somePeople
    where p.FirstName == "Larry"
    select p).Single();

Unhandled exception: System.InvalidOperationException: Sequence contains no elements
   at System.Linq.ThrowHelper.ThrowNoElementsException()
   at System.Linq.Enumerable.Single[TSource](IEnumerable`1 source)
   at Submission#60.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Of course, there are times when you expect to get more than one value but you want a specific one. The best choice is First() or FirstOrDefault().

In [58]:
(from p in somePeople where p.FirstName == "Bill" select p).FirstOrDefault()

FirstName,LastName
Bill,Gates


In [61]:
(from p in somePeople where p.FirstName == "Larry" select p).FirstOrDefault() is null

True

In [62]:
(from p in somePeople where p.FirstName == "Larry" select p).First()

Unhandled exception: System.InvalidOperationException: Sequence contains no elements
   at System.Linq.ThrowHelper.ThrowNoElementsException()
   at System.Linq.Enumerable.First[TSource](IEnumerable`1 source)
   at Submission#65.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Item 44: Avoid Modifying Bound Variables

In [70]:

public class Utilities {
    public static IEnumerable<TResult> Generate<TResult>(int number, Func<TResult> generator) {
        for (var i=0; i<number; i++) {
            yield return generator();
        }
    }
}

var index = 0;
Func<IEnumerable<int>> sequence = () => Utilities.Generate(10, () => index++);

// with an index of 20
index = 20;
foreach(int n in sequence()) {
    Console.Write(n + " ");
}
Console.WriteLine();
Console.WriteLine("Done");

// with an index of 100
index = 100;
foreach(var n in sequence()) {
    Console.Write(n + " ");
}
Console.WriteLine();

20 21 22 23 24 25 26 27 28 29 
Done
100 101 102 103 104 105 106 107 108 109 


We're effectively changing the behaviour of sequence() when changing the value of index.

In [74]:
int[] someNumbers = {0, 1, 2, 3, 4, 5};
var answers = from n in someNumbers select n * n;
foreach(var n in answers) {
    Console.Write(n + " ");
}

0 1 4 9 16 25 

***